In [15]:
from sklearn import svm

import numpy as np
import pandas as pd
import math
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
import easy_excel
from sklearn.ensemble import RandomForestClassifier

In [108]:

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
def svm_crossvalidation():
    if __name__=='__main__':
        datapath ='/home01/chenhuangrong/merge/p1_n1_H_join/p1_RPP.csv'
        train_data = pd.read_csv(datapath, header=None, index_col=None)
        X = np.array(train_data)
        Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
        Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
        Y.extend(Y2)
        Y = np.array(Y)
        parameters = {'n_estimators':range(10,200,10)}
#         parameters = {'max_depth':range(20,50,2), 'min_samples_split':range(10,60,20)}
        #parameters = {'kernel': ['rbf'], 'C':np.logspace(-5,15,20,base=2), 'gamma': np.logspace(-15,5,20,base=2)}
        clf = GridSearchCV(RandomForestClassifier(), parameters,cv=10,scoring='roc_auc')
        clf.fit(X, Y)
        print clf.best_params_
        print clf.best_score_
#         max_depth=clf.best_params_['max_depth']
#         min_samples_split=clf.best_params_['min_samples_split']
        n_estimators=clf.best_params_['n_estimators']
        y_predict=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators),X,Y,cv=10,n_jobs=-1)
        ROC_AUC_area="%0.4f"%cross_val_score(RandomForestClassifier(n_estimators=n_estimators),X,Y,cv=10,n_jobs=-1).mean()
        ACC=metrics.accuracy_score(Y,y_predict)
        precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
        F1_Score=metrics.f1_score(Y, y_predict)
        F_measure=F1_Score
        MCC=metrics.matthews_corrcoef(Y, y_predict)
        pos=TP+FN
        neg=FP+TN
        savedata=[[['randomForestClassifier'+"n_estimators:"+str(n_estimators),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
        print savedata
        easy_excel.save("crossvalidation",[str(X.shape[1])],savedata,'/home01/chenhuangrong/merge/p1_n1_H_join/random_p1_RPP.xls')

        # savafile = 'svm_crossvalidation_parameter.txt'
        # f = open(savafile, 'w+')
        # f.write("MCC:%f"%metrics.matthews_corrcoef(Y,y_predict)+'\n')
        # f.write("ACC:%f"%metrics.accuracy_score(Y,y_predict)+'\n')
        # f.write("ROC_AUC_AREA:%f"%ROC_AUC_area+'\n')
        # f.write("SN:%f,SP:%f,TP:%d:,TN:%d,FP:%d,FN:%d"%performance(Y,y_predict)+'\n')
        # f.write("ROC_AUC:%f"%metrics.roc_auc_score(Y,y_predict)+'\n')
        # f.write("C:%f,gamma:%f"%(C,gamma)+'\n')
        # f.write("F1-score:%f"%metrics.f1_score(Y,y_predict)+'\n')
        # f.write("F-measure:%f"%metrics.f1_score(Y,y_predict)+'\n')
        # f.close()
svm_crossvalidation()


{'n_estimators': 160}
0.814845376156
[[['randomForestClassifiern_estimators:160', 0.74866105585309872, 0.7531152647975078, 0.7398622800306044, 0.7398622800306044, 0.7574598316755929, 0.7486093494574471, 0.74642994982632183, 0.74642994982632183, 0.497399133415385, '0.7486', 967.0, 340.0, 317.0, 990.0, 1307.0, 1307.0]]]


In [96]:
datapath ='/home01/chenhuangrong/RFH_phisicalchemical_other/p1_n1_H_join/Rphisicalchemical_p1_n1_H_join.csv'
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.1,random_state=10)
rf = RandomForestClassifier(oob_score=True, random_state=0)
rf.fit(x_train, y_train)
print rf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn import metrics
y_train_pred = rf.predict(x_train)
y_train_predprob = rf.predict_proba(x_train)[:,1]
print u'训练集袋外分数: ', rf.oob_score_
print "训练集AUC Score: %f" % metrics.roc_auc_score(y_train, y_train_predprob)
print u'训练集准确率: ', accuracy_score(y_train, y_train_pred)
print u'训练集查准率: ', precision_score(y_train, y_train_pred)
print u'训练集召回率: ', recall_score(y_train, y_train_pred)
print u'训练集F1值: ', f1_score(y_train, y_train_pred)
print(metrics.classification_report(y_train, y_train_pred))
print(metrics.confusion_matrix(y_train, y_train_pred))
y_test_pred = rf.predict(x_test)
y_test_predprob = rf.predict_proba(x_test)[:,1]
print u'测试集准确率: ', accuracy_score(y_test, y_test_pred)
print u'测试集查准率: ', precision_score(y_test, y_test_pred)
print u'测试集召回率: ', recall_score(y_test, y_test_pred)
print u'测试集F1值: ', f1_score(y_test, y_test_pred)
print(metrics.classification_report(y_test, y_test_pred))
print(metrics.confusion_matrix(y_test, y_test_pred))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=130,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=420, n_jobs=1, oob_score=True, random_state=10,
            verbose=0, warm_start=False)
训练集袋外分数:  0.51724137931
训练集AUC Score: 0.500000
训练集准确率:  0.51724137931
训练集查准率:  0.51724137931
训练集召回率:  1.0
训练集F1值:  0.681818181818
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       126
          1       0.52      1.00      0.68       135

avg / total       0.27      0.52      0.35       261

[[  0 126]
 [  0 135]]


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


测试集准确率:  0.498087547811
测试集查准率:  0.498087547811
测试集召回率:  1.0
测试集F1值:  0.664964539007
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1181
          1       0.50      1.00      0.66      1172

avg / total       0.25      0.50      0.33      2353

[[   0 1181]
 [   0 1172]]


In [112]:
from sklearn.model_selection import GridSearchCV
datapath ='/home01/chenhuangrong/merge/p1_n1_H_join/p1_RPP.csv'
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.1,random_state=10)
param_test1 = {'min_samples_split':range(30,150,20), 'min_samples_leaf':range(10,60,10)}
gsearch1 = GridSearchCV(estimator=RandomForestClassifier(n_estimators=180, max_features='sqrt',min_samples_split=30,max_depth=3, min_samples_leaf=20,oob_score=True, random_state=10),param_grid=param_test1, scoring='roc_auc', cv=5)
gsearch1.fit(x_train, y_train)
print gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

{'rank_test_score': array([ 6, 12, 20, 15, 16, 29,  1,  2, 10,  7, 14, 26,  8,  8, 11, 13, 19,
       27,  3,  3,  3, 18, 17, 30, 21, 21, 21, 21, 21, 28], dtype=int32), 'std_score_time': array([ 0.07012594,  0.08230116,  0.1159338 ,  0.09874819,  0.05966545,
        0.05290366,  0.07467623,  0.07570851,  0.11859488,  0.06485003,
        0.17946886,  0.04967699,  0.11948111,  0.16650474,  0.15593188,
        0.07388363,  0.06560802,  0.06552192,  0.03134922,  0.11241518,
        0.06511331,  0.1052271 ,  0.05263414,  0.07671741,  0.04478254,
        0.07920863,  0.10605121,  0.15978561,  0.05288905,  0.12063068]), 'split2_test_score': array([ 0.89333333,  0.89777778,  0.87555556,  0.88296296,  0.8962963 ,
        0.8562963 ,  0.89037037,  0.89481481,  0.86814815,  0.88148148,
        0.88888889,  0.84888889,  0.86814815,  0.86814815,  0.88      ,
        0.88      ,  0.88740741,  0.85925926,  0.88296296,  0.88296296,
        0.88296296,  0.88      ,  0.88      ,  0.84296296,  0.84888889

In [3]:

import os
os.chdir("/home01/chenhuangrong/")
import easy_excel
import pandas as pd
import mlens
print os.getcwd()


/home01/chenhuangrong


In [104]:
easy_excel.save("rossvalidation",[str(X.shape[1]),str(X.shape[1]),str(X.shape[1])],[[['randomForestClassifiermax_depth20min_samples_split10', 0.7517214996174445, 0.7542503863987635, 0.7467482785003825, 0.7467482785003825, 0.7566947207345065, 0.7517050485787763, 0.75048058439061893, 0.75048058439061893, 0.50346790432201705, '0.7517', 976.0, 331.0, 318.0, 989.0, 1307.0, 1307.0],['randomForestClassifiermax_depth20min_samples_split10', 0.7517214996174445, 0.7542503863987635, 0.7467482785003825, 0.7467482785003825, 0.7566947207345065, 0.7517050485787763, 0.75048058439061893, 0.75048058439061893, 0.50346790432201705, '0.7517', 976.0, 331.0, 318.0, 989.0, 1307.0, 1307.0],['randomForestClassifiermax_depth20min_samples_split10', 0.7517214996174445, 0.7542503863987635, 0.7467482785003825, 0.7467482785003825, 0.7566947207345065, 0.7517050485787763, 0.75048058439061893, 0.75048058439061893, 0.50346790432201705, '0.7517', 976.0, 331.0, 318.0, 989.0, 1307.0, 1307.0]]],'/home01/chenhuangrong/RFH_phisicalchemical_other/p1_n1_H_join/random.xls')

True

In [ ]:
from sklearn import svm

import numpy as np
import pandas as pd
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
def svm_crossvalidation():
    if __name__=='__main__':
        datapath ='/home01/chenhuangrong/merge/p1_n1_H_join/BPbayes_PSTNP_p1_RPP.csv'
        train_data = pd.read_csv(datapath, header=None, index_col=None)
        X = np.array(train_data)
        Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
        Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
        Y.extend(Y2)
        Y = np.array(Y)
        svc = svm.SVC()
        #parameters = {'kernel':['rbf'], 'C': np.linspace(32750,32799,100), 'gamma': np.linspace(0.00001,0.00009,20)}
        parameters = {'kernel': ['rbf'], 'C':np.logspace(-5,15,20,base=2), 'gamma': np.logspace(-30,-10,20,base=2)}
        clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='roc_auc')
        clf.fit(X, Y)
        C=clf.best_params_['C']
        print clf.best_score_
        gamma=clf.best_params_['gamma']
        y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1)
        ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
        ACC=metrics.accuracy_score(Y,y_predict)
        precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
        F1_Score=metrics.f1_score(Y, y_predict)
        F_measure=F1_Score
        MCC=metrics.matthews_corrcoef(Y, y_predict)
        pos=TP+FN
        neg=FP+TN
        savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
        print savedata
        easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home01/chenhuangrong/merge/p1_n1_H_join/test2.xls')

        # savafile = 'svm_crossvalidation_parameter.txt'
        # f = open(savafile, 'w+')
        # f.write("MCC:%f"%metrics.matthews_corrcoef(Y,y_predict)+'\n')
        # f.write("ACC:%f"%metrics.accuracy_score(Y,y_predict)+'\n')
        # f.write("ROC_AUC_AREA:%f"%ROC_AUC_area+'\n')
        # f.write("SN:%f,SP:%f,TP:%d:,TN:%d,FP:%d,FN:%d"%performance(Y,y_predict)+'\n')
        # f.write("ROC_AUC:%f"%metrics.roc_auc_score(Y,y_predict)+'\n')
        # f.write("C:%f,gamma:%f"%(C,gamma)+'\n')
        # f.write("F1-score:%f"%metrics.f1_score(Y,y_predict)+'\n')
        # f.write("F-measure:%f"%metrics.f1_score(Y,y_predict)+'\n')
        # f.close()
svm_crossvalidation()